# Neural Networks 101 with `gluon`

Gluon으로 간단한 네트워크를 만들어 보도록 한다. 

1. 네트워크 정의 
1. 파라메터 초기화 
1. 각각 배치 입력에 따라... 
  1. 전방입력(feedforword)
  1. 네트워크 출력과 레이블 사이의 손실 계산 
  1. 그레디언트를 계산하기 위해 backpropagation 수행 
  1. 파라메터 업데이트

In [2]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd, gluon

어디에서 계산이 될지 컨텍스트 정의 

In [3]:
data_ctx = mx.cpu()
# model_ctx = mx.cpu()
model_ctx = mx.gpu(0)

## `Block`s in `gluon`

`gluon.Block`은 모델을 구성하는 기본 단위이며 `Block`을 상속받아 생성된다. 

```
class Net(gluon.Block):
    [...]  # We cover the __init__ function later

    # One or more NDArrays can be passed to `forward`
    def forward(self, x):
        # Computation
        # Do something with your data x to compute y
        return y
```

### `Block`의 존재 이유 

- `gluon.Block`은 학습 파라메터를 숨기고 사용자가 입력과 출력에  신경쓸 수 있도록 함 

### Using `Block`s

- Glon은 많은 `Block`을 제공하고 있음 
- 예를 들어 `Dense` 레이어는... 
- `Dense`는 `output = activation(dot(input, weight) + bias)`를 구현하고 있음 


```
    def hybrid_forward(self, F, x, weight, bias=None):
        act = F.FullyConnected(x, weight, bias, no_bias=bias is None, num_hidden=self._units,
                               flatten=self._flatten, name='fwd')
        if self.act is not None:
            act = self.act(act)
    return act
```


### 간단한 `Block`

In [14]:
net = gluon.nn.Dense(1, in_units=2)  # 단일 출력, 두개의 입력 유닛

블록은 파라메터를 숨기고 있지만, 아래와 같은 명령어로 확인해 볼 수 있음 

In [15]:
print(net.weight)
print(net.bias)

Parameter dense1_weight (shape=(1, 2), dtype=float32)
Parameter dense1_bias (shape=(1,), dtype=float32)


`collect_params()`은 블록이 가지고 있는 모든 파라메터를 리턴한다. 
    

In [16]:
net.collect_params()

dense1_ (
  Parameter dense1_weight (shape=(1, 2), dtype=float32)
  Parameter dense1_bias (shape=(1,), dtype=float32)
)

`gluon.parameter.ParameterDict`타입

In [17]:
type(net.collect_params())

mxnet.gluon.parameter.ParameterDict

모든 `Block`객체는 학습 전 초기화를 시켜야 되며, 동시에 파라메터가 어떠한 컨텍스트에 존재해야 되는지 명시해야 된다. 


In [18]:
net.initialize(mx.initializer.Uniform(), ctx=model_ctx)

초기화 된 이후엔 직접적으로 파라메터 값에 대한 접근이 가능해진다. 

In [19]:
print(net.weight.data())
print(net.bias.data())


[[0.0301265  0.04819721]]
<NDArray 1x2 @gpu(0)>

[0.]
<NDArray 1 @gpu(0)>


### 옵티마이저

In [20]:
square_loss = gluon.loss.L2Loss()

In [21]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.0001})

### 데이터

In [23]:
num_inputs = 2
num_outputs = 1
num_examples = 10000

In [24]:
X = nd.random_normal(shape=(num_examples, num_inputs))
noise = 0.01 * nd.random_normal(shape=(num_examples, ))

In [28]:
def real_fn(X):
    return 2 * X[:, 0] - 3.4 * X[:, 1] + 4.2

y = real_fn(X) + noise

In [29]:
print(X)


[[ 1.0434405   1.1839255 ]
 [ 1.8917114  -1.2347414 ]
 [-1.771029   -0.45138445]
 ...
 [ 0.08873925 -0.45150325]
 [-0.13049959  0.15614532]
 [-0.22753173 -0.19928493]]
<NDArray 10000x2 @cpu(0)>


In [30]:
print(y)


[ 2.2621982 12.19003    2.1846452 ...  5.9032497  3.4229667  4.4221096]
<NDArray 10000 @cpu(0)>


### 배치

In [31]:
batch_size = 4
train_data = gluon.data.DataLoader(
    gluon.data.ArrayDataset(X, y), batch_size=batch_size, shuffle=True)

### 배치 학습

In [32]:
epochs = 10
num_batches = num_examples / batch_size
print(num_batches)

2500.0


In [33]:
def train_loop(epochs):
    for e in range(epochs):
        cumulative_loss = 0
        for i, (data, label) in enumerate(train_data):
            data = data.as_in_context(model_ctx)
            label = label.as_in_context(model_ctx)
            with autograd.record():
                output = net(data)
                loss = square_loss(output, label)
            loss.backward()
            trainer.step(batch_size)
            cumulative_loss += nd.mean(loss).asscalar()
        print("Epoch %s, loss: %.4f" % (e, cumulative_loss / num_examples))

In [34]:
train_loop(epochs)

Epoch 0, loss: 3.2904
Epoch 1, loss: 1.9942
Epoch 2, loss: 1.2087
Epoch 3, loss: 0.7326
Epoch 4, loss: 0.4440
Epoch 5, loss: 0.2691
Epoch 6, loss: 0.1631
Epoch 7, loss: 0.0989
Epoch 8, loss: 0.0599
Epoch 9, loss: 0.0363


## 파라메터 확인

In [36]:
params = net.collect_params() 
for param in params.values():
    print(param.name,param.data())

dense1_weight 
[[ 1.8478074 -3.113041 ]]
<NDArray 1x2 @gpu(0)>
dense1_bias 
[3.8561728]
<NDArray 1 @gpu(0)>
